In [2]:
!pip install -U bitsandbytes
!pip install transformers==4.36.2
!pip install -U peft
!pip install -U accelerate
!pip install -U trl
!pip install datasets==2.16.0
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 35.6 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.37.2
    Uninstalling transformers-4.37.2:
      Successfully uninstalled transformers-4.37.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index-llms-nvidia-tensorrt 0.1.2 requires transformers<5.0.0,>=4.37.0, but you have transformers 4.36.2 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging
# from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os,torch
from datasets import load_dataset
from trl import SFTTrainer
import pandas as pd
import pyarrow as pa
import pyarrow.dataset as ds
import pandas as pd
from datasets import Dataset
import re

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3553 in run_code        │
│                                                                                                  │
│   3550 │   │   │   │   elif async_ :                                                             │
│   3551 │   │   │   │   │   await eval(code_obj, self.user_global_ns, self.user_ns)               │
│   3552 │   │   │   │   else:                                                                     │
│ ❱ 3553 │   │   │   │   │   exec(code_obj, self.user_global_ns, self.user_ns)                     │
│   3554 │   │   │   finally:                                                                      │
│   3555 │   │   │   │   # Reset our crash handler in place                                        │
│   3556 │   │   │   │   sys.excepthook = old_excepthook                                           │
│ in <cell line: 5>:5                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/trl/__init__.py:7 in <module>                            │
│                                                                                                  │
│    4                                                                                             │
│    5 from .core import set_seed                                                                  │
│    6 from .environment import TextEnvironment, TextHistory                                       │
│ ❱  7 from .extras import BestOfNSampler                                                          │
│    8 from .import_utils import (                                                                 │
│    9 │   is_bitsandbytes_available,                                                              │
│   10 │   is_diffusers_available,                                                                 │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/trl/extras/__init__.py:16 in <module>                    │
│                                                                                                  │
│   13 # WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.                  │
│   14 # See the License for the specific language governing permissions and                       │
│   15 # limitations under the License.                                                            │
│ ❱ 16 from .best_of_n_sampler import BestOfNSampler                                               │
│   17                                                                                             │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/trl/extras/best_of_n_sampler.py:7 in <module>            │
│                                                                                                  │
│     4 from transformers import GenerationConfig, PreTrainedTokenizer, PreTrainedTokenizerFast    │
│     5                                                                                            │
│     6 from ..core import set_seed                                                                │
│ ❱   7 from ..models import SUPPORTED_ARCHITECTURES, PreTrainedModelWrapper                       │
│     8                                                                                            │
│     9                                                                                            │
│    10 class BestOfNSampler:                                                                      │
│                                                            

In [5]:
train_set=pd.read_csv("/content/drive/MyDrive/final_train_undersampled.csv")
test_set=pd.read_csv("/content/drive/MyDrive/final_test_undersampled.csv")
eval_set=pd.read_csv("/content/drive/MyDrive/final_val_undersampled.csv")

NameError: name 'pd' is not defined

In [5]:
def create_finetuning_prompt(sample):
    resume=sample['Resume']
    job_description=sample['JobDescription']
    score=sample['Score']
    bos_token = "<s>"
    eos_token = "</s>"

    # Instruction for the model to analyze the resume's relevance to the job description
    instruction = "[INST] You are a seasoned job recruiter at a leading tech company, specializing in identifying top talent for technology-focused positions. Your expertise includes a deep understanding of the skills, experiences, and qualities that make a candidate a perfect fit for challenging and innovative tech roles. Given the resume and job description below, analyze the compatibility of the candidate's qualifications with the job requirements and provide an ATS (Applicant Tracking System) relevance score expressed as a percentage, where 100% indicates a perfect match. This score reflects the degree to which the candidate's experience, skills, and education align with the job's requirements as defined by the recruiter"

    # Machine-generated response example, encapsulating the expected outcome or analysis.
    # Replace 'machine_response' with an actual example response that your model should learn to generate.
    response = f"The relevance/ATS score of the given resume to the job description is {score}%."

    # Concatenating the elements to form the full prompt
    full_prompt = f"{bos_token}\n\n{instruction}\nDont give any additional analysis,give the ATS score and the response should be in this format :The relevance/ATS score of the given resume to the job description is X%.\n\n### Resume:\n{resume}\n\n### Job Description:\n{job_description}\n\n [/INST]\n\n### Response:\n{response}{eos_token}"

    return full_prompt


In [6]:
train_set.head()

,Resume,JobDescription,Score,ScoreCategory
0,RECRUITING AND HR MANAGER Summa...,designups is a nashville based design and inte...,37,Medium
1,â¢ Good logical and analytical skills â¢ Pos...,job description\n\nthe patient advocate will p...,9,Low
2,KEY COMPETENCIES â¶Multi - Operations Managem...,company description\n\nat servicenow our techn...,57,Medium
3,PROJECT ADMINISTRATOR \tENGINEERING ...,job category retail\n\nrequisition number \n\n...,22,Low
4,HR SPECIALIST Summary E...,description this position is a hybrid positio...,57,Medium


In [7]:
train_set = train_set.sample(frac=1, random_state=42).reset_index(drop=True)

In [5]:
train_set.head()

,Resume,JobDescription,Score,ScoreCategory
0,RECRUITING AND HR MANAGER Summa...,designups is a nashville based design and inte...,37,Medium
1,â¢ Good logical and analytical skills â¢ Pos...,job description\n\nthe patient advocate will p...,9,Low
2,KEY COMPETENCIES â¶Multi - Operations Managem...,company description\n\nat servicenow our techn...,57,Medium
3,PROJECT ADMINISTRATOR \tENGINEERING ...,job category retail\n\nrequisition number \n\n...,22,Low
4,HR SPECIALIST Summary E...,description this position is a hybrid positio...,57,Medium


In [10]:
create_finetuning_prompt(train_set.iloc[0])

'<s>\n\n[INST] You are a seasoned job recruiter at a leading tech company, specializing in identifying top talent for technology-focused positions. Your expertise includes a deep understanding of the skills, experiences, and qualities that make a candidate a perfect fit for challenging and innovative tech roles. Given the resume and job description below, analyze the compatibility of the candidate\'s qualifications with the job requirements and provide an ATS (Applicant Tracking System) relevance score expressed as a percentage, where 100% indicates a perfect match. This score reflects the degree to which the candidate\'s experience, skills, and education align with the job\'s requirements as defined by the recruiter\nDont give any additional analysis,give the ATS score and the response should be in this format :The relevance/ATS score of the given resume to the job description is X%.\n\n### Resume:\n         RECRUITING AND HR MANAGER       Summary     Experienced Corporate Recruiter w

In [8]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.2",  # Updated to v0.2
    device_map='auto',
    quantization_config=nf4_config,
    use_cache=False
)

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")  # Make sure to match the version

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"


config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [9]:
def generate_response(prompt, model):
  encoded_input = tokenizer(prompt,  return_tensors="pt", add_special_tokens=True)
  model_inputs = encoded_input.to('cuda')

  generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id)

  decoded_output = tokenizer.batch_decode(generated_ids)

  return decoded_output[0].replace(prompt, "")

In [10]:
prompt="<s> [INST] You are a seasoned job recruiter at a leading tech company, specializing in identifying top talent for technology-focused positions. Your expertise includes a deep understanding of the skills, experiences, and qualities that make a candidate a perfect fit for challenging and innovative tech roles. Given the resume and job description below, analyze the compatibility of the candidate's qualifications with the job requirements and provide an ATS (Applicant Tracking System) relevance score expressed as a percentage, where 100% indicates a perfect match. This score reflects the degree to which the candidate's experience, skills, and education align with the job's requirements as defined by the recruiter.\nDont give any additional analysis,give the ATS score and the response should be in this format :The relevance/ATS score of the given resume to the job description is X%.\n\n### Resume:\n\nJohn Doe\n\nEducation:\nB.Sc. in Computer Science, XYZ University (2015-2019)\n\nExperience:\nSoftware Developer at TechCorp (2019-Present)\nKey achievements include:\n- Leading the development of a scalable web application that increased company revenue by 20%.\n- Contributing to the improvement of the continuous integration pipeline, reducing code deployment time by 40%.\n\nSkills:\n- Proficient in Python, Java, and JavaScript\n- Experienced with cloud services (AWS, GCP)\n- Agile and Scrum methodologies\n\nCertifications:\nCertified Kubernetes Administrator (2020)\nAWS Certified Developer (2021)\n\nReferences:\nAvailable upon request.\n\n### Job Description:\n\n\nSenior Software Developer\nXYZ Innovations Inc.\n\nWe are seeking a senior software developer with a strong background in modern web application development and cloud technologies to join our dynamic team. The ideal candidate will be an integral part of our efforts to innovate and deliver high-quality software solutions.\n\nResponsibilities:\n- Develop and maintain scalable web applications.\n- Collaborate with cross-functional teams to bring new features to life.\n- Optimize applications for maximum speed and scalability.\n\nRequirements:\n- Bachelor's degree in Computer Science or related field.\n- 5+ years of experience in software development.\n- In-depth knowledge of Python, Java, and JavaScript.\n- Proven experience with AWS and Kubernetes.\n- Familiarity with CI/CD pipelines and Agile methodologies.\n\n[/INST]"


In [14]:
#prompt="<s> [INST] You are a seasoned job recruiter at a leading tech company, specializing in identifying top talent for technology-focused positions. Your expertise includes a deep understanding of the skills, experiences, and qualities that make a candidate a perfect fit for challenging and innovative tech roles. Given the resume and job description below, analyze the compatibility of the candidate's qualifications with the job requirements and provide an ATS (Applicant Tracking System) relevance score expressed as a percentage, where 100% indicates a perfect match. This score reflects the degree to which the candidate's experience, skills, and education align with the job's requirements as defined by the recruiter.\n\n### Resume:\nEducation Details \r\nJanuary 2005 S.S.C  Baramati, Maharashtra M.E.S. Highschool\r\n B.E. Civil Engineering Pune, Maharashtra Singhad Institute Of Technology And Science\r\ncivil site engineer \r\n\r\nCivil engineer\r\nSkill Details \r\nSite Engineer- Exprience - 60 monthsCompany Details \r\ncompany - Vasundhara nirmiti properties\r\ndescription - Civil engineer\r\ncompany - shri balaji housing company\r\ndescription - Execution of all civil work\r\ncompany - Ganesh construction\r\ndescription - Site engineer\n\n### Job Description:\n\n\nplease read this job posting in its entirety\nwho may apply\nthis special agent sa posting is open to all qualified candidates within the united states us and its territories the fbi will use this to fill multiple vacancies around the country\njob summary\nthe mission of the federal bureau of investigation fbi is to protect the american people and uphold the constitution of the united states\nabout us\nwere looking for people with unique skills experiences and backgrounds to become special agents\xa0the one thing every sa shares is a belief in the mission of the fbi they have the drive to help keep our nation safe become part of something bigger than themselves and take pride in making a difference in the communities they serve\nspecial agents are\ndriven to find solutions to all types of problemscompassionate and respectful to everyone they meetadaptable to a variety of situations and able to keep a calm demeanor in the face of challengeswellspoken wellwritten and enjoy meeting speaking and collaborating with people from different backgroundsphysically fit and able to pass the physical fitness test pftkey requirements\nyou must meet the following requirements\xa0at the time of application\nplease click on the link and go to the listed page number if applicable for further explanation\nbe a us citizenbe between  and  years of age exceptions\xa0have a\xa0bachelors degree or masters degree\xa0 from a usaccredited college or university you should not apply if you have not received your\xa0bachelors degreehave at least two years of\xa0fulltime professional work experience\xa0 or one year if you have an advanced degree masters degree doctorate degreepossess a valid drivers license and have six months of driving experiencemeet the\xa0physical requirementsmeet all\xa0fbi employment eligibility requirementsbe able to obtain a top secret sensitive compartmented information sci clearanceif you are currently active duty in the military you must be within  months of completing your service before submitting your applicationbe available for\xa0assignment\xa0 to any of the fbis  field officesmajor duties\nthe sa position requires significant commitment and dedication\xa0sas must\nadhere to the highest standards of conduct especially in maintaining honesty and integritywork a minimum of  hours per week which may include irregular hours and be oncall  including holidays and weekendsmaintain a level of fitness necessary to effectively respond to lifethreatening situations on the jobcarry a firearm and be willing to use deadly force if necessarybe willing and able to participate in arrests execution of search warrants and other dangerous assignmentsbefore you apply\nyou should review the\xa0special agent selection process all you need to know to apply\xa0guide before you begin the application process\xa0this guide provides helpful information about processing for the special agent positionabout the application process\nthe application process consists of the belowlisted steps and you must successfully complete each step of the process\xa0those who meet the screening qualifications will be invited to complete the phase i exam via an email notification\nstep \xa0application and screening\xa0step \xa0phase i\xa0teststep \xa0meet and greet sessionreview process\xa0step \xa0phase ii teststep \xa0conditional appointment offer\xa0step \xa0background investigation\xa0step \xa0official physical fitness test pft\xa0step \xa0basic field training course bftcfinish\xa0career placement\xa0required documentation for your application\nplease review and submit all required documents below with your application\xa0if you do not submit all required documents your application will not be processed and will be discontinued\nall special agent applicants\nofficial or unofficial college transcriptsa resume formatted in the\xa0federal resume template\xa0that meets all guidelines outlined in the\xa0federal resume guideplease note\ndo not\xa0attach a cover letterwhen scanning and uploading your documents please make sure that all pages are readable and facing upright in the same directiondo not\xa0upload any picture files jpeg tiff png bmpadditional information\nsalary and compensation\xa0new special agents called new agent trainees or nats are paid on the\xa0gl schedule for federal law enforcement officers\xa0leo beginning as a gl step  while in training at quantico in addition to the base salary nats will also get locality pay for the washington dc area plus availability pay which is  of the base and locality pay the calculation is  x \nupon graduation from training your salary will be adjusted according to your field office assignment and\xa0law enforcement officers availability pay check the opm pay calculator to see locality pay for a specific geographic area upon completion of a twoyear probationary period special agents will transition to the general schedule gs pay scale most special agents are able to achieve a gs level within five  years\nreasonable accommodations\xa0the fbi provides reasonable accommodations to qualified individuals with disabilities\xa0if you believe you have a disability that will require special arrangements during the hiring process you may request a reasonable accommodation at any time by notifying your applicant coordinator\xa0each test in the hiring process is evaluated differently\xa0therefore you must submit a reasonable accommodation request for each test if needed the decision to grant the requested accommodation will be made on a casebycase basis\ncovid vaccination requirement\non september   executive orders were signed by the president requiring coronavirus disease  vaccination for federal employees all individuals who enter government service after november   need to be fully vaccinated prior to their start date except in limited circumstances where an employee is entitled to a reasonable accommodation pursuant to a sincerely held religious belief or medical disability an individual is considered fully vaccinated two weeks after the second injection in a twodose series pfizerbiontech or moderna or two weeks after the single injection option johnson  johnsonjanssen all employees are required to attest to their vaccination status and upload documentation of proof using the fbis official and secure medical system of record for more information regarding where to obtain a vaccine at no cost to you please visit\xa0\xa0\nprior to submitting a reasonable accommodation request relating to the covid vaccine mandate an applicant should ensure they meet the following  in possession of a conditional appointment offer cao or conditional job offer cjo  passed the fbi polygraph and  scheduled for an applicant fitness for duty medical clearance exam if these criteria are met an applicant may proceed with submitting a reasonable accommodation request to the oeeoa reasonable accommodation program\xa0\nnote all individuals applying for a position with a medical requirement must provide proof of covid vaccination status or an approved exemption applicants for this position will not receive a final job offer until the medical clearance has been obtained\n\n[/INST]"
generate_response(prompt,model)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1518: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


"<s><s>  [INST] You are a seasoned job recruiter at a leading tech company, specializing in identifying top talent for technology-focused positions. Your expertise includes a deep understanding of the skills, experiences, and qualities that make a candidate a perfect fit for challenging and innovative tech roles. Given the resume and job description below, analyze the compatibility of the candidate's qualifications with the job requirements and provide an ATS (Applicant Tracking System) relevance score expressed as a percentage, where 100% indicates a perfect match. This score reflects the degree to which the candidate's experience, skills, and education align with the job's requirements as defined by the recruiter.\nDont give any additional analysis,give the ATS score and the response should be in this format :The relevance/ATS score of the given resume to the job description is X%.\n\n### Resume:\n\nJohn Doe\n\nEducation:\nB.Sc. in Computer Science, XYZ University (2015-2019)\n\nExpe

In [11]:
from peft import AutoPeftModelForCausalLM, LoraConfig, get_peft_model, prepare_model_for_kbit_training

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM"
)

In [12]:
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

In [13]:
from transformers import TrainingArguments

args = TrainingArguments(
  output_dir = "mistral_instruct_generation",
  num_train_epochs=4,
  # max_steps = 100, # comment out this line if you want to train in epochs
  per_device_train_batch_size = 4,
  warmup_steps = 0.03,
  logging_steps=10,
  save_strategy="epoch",
  #evaluation_strategy="epoch",
  evaluation_strategy="steps",
  eval_steps=20, # comment out this line if you want to evaluate at the end of each epoch
  learning_rate=2e-4,
  bf16=True,
  lr_scheduler_type='constant',
)

In [15]:
from datasets import Dataset

# Convert the train and test sets from Pandas DataFrame to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_set)
eval_dataset = Dataset.from_pandas(eval_set)

from trl import SFTTrainer

max_seq_length = 2048

trainer = SFTTrainer(
  model=model,
  peft_config=peft_config,
  max_seq_length=max_seq_length,
  tokenizer=tokenizer,
  packing=True,
  formatting_func=create_finetuning_prompt,
  args=args,
  train_dataset=train_dataset,
  eval_dataset=eval_dataset
)

In [19]:
trainer.train()

Step,Training Loss,Validation Loss
20,1.880400,1.883357
40,1.836400,1.863066
60,1.836300,1.854720
80,1.831200,1.829822
100,1.764800,1.810174
120,1.619700,1.788791
140,1.686900,1.788690
160,1.563700,1.767153
180,1.692100,1.747588
200,1.588300,1.730532


Checkpoint destination directory mistral_instruct_generation/checkpoint-118 already exists and is non-empty.Saving will proceed but saved results may be invalid.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for mor

TrainOutput(global_step=472, training_loss=1.5192890157133847, metrics={'train_runtime': 2833.3814, 'train_samples_per_second': 0.666, 'train_steps_per_second': 0.167, 'total_flos': 1.659488311569285e+17, 'train_loss': 1.5192890157133847, 'epoch': 4.0})

In [34]:
trainer.save_model("mistral_instruct_generation")
# trainer.save_model("/content/drive/MyDrive/mistral_ATSscore_generation")
# trainer.mode.save_config("/content/drive/MyDrive/mistral_ATSscore_generation/config.json")

AttributeError: 'SFTTrainer' object has no attribute 'mode'

In [21]:
!pip install huggingface-hub -qU

In [22]:
from huggingface_hub import notebook_login

notebook_login()

In [23]:
trainer.push_to_hub("SlyGoblin/mistral_ATSscore_generation")

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

events.out.tfevents.1708089687.a98c3daebe8f.1676.0:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/109M [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/4.73k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/SlyGoblin/mistral_instruct_generation/commit/c2019c4adc7a67173a049e7b04b0e30e266c2d8d', commit_message='SlyGoblin/mistral_ATSscore_generation', commit_description='', oid='c2019c4adc7a67173a049e7b04b0e30e266c2d8d', pr_url=None, pr_revision=None, pr_num=None)

In [26]:
merged_model = model.merge_and_unload()

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:249: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [44]:
merged_model.save_model("MyModel")

AttributeError: 'MistralForCausalLM' object has no attribute 'save_model'

In [4]:
def create_input(sample):
    resume=sample['Resume']
    job_description=sample['JobDescription']
    score=sample['Score']
    bos_token = "<s>"
    eos_token = "</s>"

    # Instruction for the model to analyze the resume's relevance to the job description
    instruction = "[INST] You are a seasoned job recruiter at a leading tech company, specializing in identifying top talent for technology-focused positions. Your expertise includes a deep understanding of the skills, experiences, and qualities that make a candidate a perfect fit for challenging and innovative tech roles. Given the resume and job description below, analyze the compatibility of the candidate's qualifications with the job requirements and provide an ATS (Applicant Tracking System) relevance score expressed as a percentage, where 100% indicates a perfect match. This score reflects the degree to which the candidate's experience, skills, and education align with the job's requirements as defined by the recruiter"

    full_prompt = f"{bos_token}\n\n{instruction}\nDont give any additional analysis,give the ATS score and the response should be in this format :The relevance/ATS score of the given resume to the job description is X%.\n\n### Resume:\n{resume}\n\n### Job Description:\n{job_description}\n\n [/INST]{eos_token}"

    return full_prompt


In [25]:
# prompt="<s> [INST] You are a seasoned job recruiter at a leading tech company, specializing in identifying top talent for technology-focused positions. Your expertise includes a deep understanding of the skills, experiences, and qualities that make a candidate a perfect fit for challenging and innovative tech roles. Given the resume and job description below, analyze the compatibility of the candidate's qualifications with the job requirements and provide an ATS (Applicant Tracking System) relevance score expressed as a percentage, where 100% indicates a perfect match. This score reflects the degree to which the candidate's experience, skills, and education align with the job's requirements as defined by the recruiter.\n\n### Resume:\n\nJohn Doe\n\nEducation:\nB.Sc. in Computer Science, XYZ University (2015-2019)\n\nExperience:\nSoftware Developer at TechCorp (2019-Present)\nKey achievements include:\n- Leading the development of a scalable web application that increased company revenue by 20%.\n- Contributing to the improvement of the continuous integration pipeline, reducing code deployment time by 40%.\n\nSkills:\n- Proficient in Python, Java, and JavaScript\n- Experienced with cloud services (AWS, GCP)\n- Agile and Scrum methodologies\n\nCertifications:\nCertified Kubernetes Administrator (2020)\nAWS Certified Developer (2021)\n\nReferences:\nAvailable upon request.\n\n### Job Description:\n\n\nSenior Software Developer\nXYZ Innovations Inc.\n\nWe are seeking a senior software developer with a strong background in modern web application development and cloud technologies to join our dynamic team. The ideal candidate will be an integral part of our efforts to innovate and deliver high-quality software solutions.\n\nResponsibilities:\n- Develop and maintain scalable web applications.\n- Collaborate with cross-functional teams to bring new features to life.\n- Optimize applications for maximum speed and scalability.\n\nRequirements:\n- Bachelor's degree in Computer Science or related field.\n- 5+ years of experience in software development.\n- In-depth knowledge of Python, Java, and JavaScript.\n- Proven experience with AWS and Kubernetes.\n- Familiarity with CI/CD pipelines and Agile methodologies.\n\n[/INST]"
prompt=create_input(test_set.iloc[0])

In [28]:
test_set.iloc[0]

Resume            COMPUTER SKILLS & SOFTWARE KNOWLEDGE MS-Power ...
JobDescription    hello seekers\n\nwere a web agency based in ti...
Score                                                            53
ScoreCategory                                                Medium
Name: 0, dtype: object

In [27]:
generate_response(prompt,merged_model)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1518: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs fo

"<s><s> \n\n[INST] You are a seasoned job recruiter at a leading tech company, specializing in identifying top talent for technology-focused positions. Your expertise includes a deep understanding of the skills, experiences, and qualities that make a candidate a perfect fit for challenging and innovative tech roles. Given the resume and job description below, analyze the compatibility of the candidate's qualifications with the job requirements and provide an ATS (Applicant Tracking System) relevance score expressed as a percentage, where 100% indicates a perfect match. This score reflects the degree to which the candidate's experience, skills, and education align with the job's requirements as defined by the recruiter\nDont give any additional analysis,give the ATS score and the response should be in this format :The relevance/ATS score of the given resume to the job description is X%.\n\n### Resume:\nCOMPUTER SKILLS & SOFTWARE KNOWLEDGE MS-Power Point, MS - Office, C, Protius (PCB Des

In [32]:
import time

# Generate the prompt for the first row
prompt = create_input(test_set.iloc[0])

# Start measuring time
start_time = time.time()

# Generate response for the prompt
response = generate_response(prompt, merged_model)

# Stop measuring time
end_time = time.time()

# Calculate the inference time
inference_time = end_time - start_time

print(f"Inference time for 1 row: {inference_time:.2f} seconds")


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Inference time for 1 row: 1089.26 seconds


In [41]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [1]:
%pip install llama-index-llms-nvidia-tensorrt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 631.0/631.0 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 103.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 88.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 56.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 66.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 99.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [3]:
!pip install tensorrt_llm==0.7.0 --extra-index-url https://pypi.nvidia.com --extra-index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com, https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 340.3/340.3 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 851.8/851.8 kB 42.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 34.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 46.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.9/327.9 kB 37.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 56.2

In [ ]:
from llama_index.llms.nvidia_tensorrt import LocalTensorRTLLM

llm = LocalTensorRTLLM(
    model_path="",
    engine_name="llama_float16_tp1_rank0.engine",
    tokenizer_dir="",
    completion_to_prompt=completion_to_prompt,
)

In [ ]:
import time

# Generate the prompt for the first row
prompt = create_input(test_set.iloc[0])

# Start measuring time
start_time = time.time()

resp = llm.complete(prompt)
# Stop measuring time
end_time = time.time()

# Calculate the inference time
inference_time = end_time - start_time

print(f"Inference time for 1 row: {inference_time:.2f} seconds")